In [ ]:
import boto3
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from decimal import Decimal
from s3fs import S3FileSystem
from datetime import datetime
from dotenv import load_dotenv

In [ ]:
load_dotenv()


# 1. Create Bucket

In [ ]:
session = boto3.Session(profile_name="mentha")
s3_client = session.client("s3")

In [ ]:
# s3_client.list_buckets()["Buckets"]

# s3_client.create_bucket(Bucket="mentha-athena-datastore",
#                         ACL="public-read",
#                         CreateBucketConfiguration={
#                             "LocationConstraint": "ap-northeast-2"
#                         }
# )

s3_client.list_buckets()["Buckets"]

# 2. Convert csv(local) to parquet(s3)

## 2.1. Read csv

In [ ]:
def decimal_from_value(value):
    if value == "null":
        return pd.NA
    else:
        return Decimal(value)

event_metadata = {
    "column_list": ["identity_adid", "os", "model", "country", "event_name", "log_id", "server_datetime", "quantity", "price"],
    "dtype_dict": {
        "identity_adid": "str",
        "os": "str",
        "model": "str",
        "country": "str",
        "event_name": "str",
        "log_id": "str",
        "quantity": "Int64",
        "server_datetime": "str"
    },
    "convert_dict": {
        "price": decimal_from_value
    },
}

attribution_metadata = {
    "column_list": ["partner", "campaign", "server_datetime", "tracker_id", "log_id", "attribution_type", "identity_adid"],
    "dtype_dict": {
        "partner": "str",
        "campaign": "str",
        "tracker_id": "str",
        "log_id": "str",
        "attribution_type": "Int64",
        "identity_adid": "str",
        "server_datetime": "str"
    }
}

In [ ]:
df_event = pd.read_csv("data/event.csv",
                       names=event_metadata["column_list"],
                       dtype=event_metadata["dtype_dict"],
                       converters=event_metadata["convert_dict"]
)

In [ ]:
df_attribution = pd.read_csv("data/attribution.csv",
                             names=attribution_metadata["column_list"],
                             dtype=attribution_metadata["dtype_dict"],
)

### server_datetime 에러 케이스 확인
- event 테이블에서 "0001-01-01 00:00:00.0"인 경우 Null로 대체하자

In [ ]:
df_event["server_datetime"] = pd.to_datetime(df_event["server_datetime"], errors = "coerce")
df_attribution["server_datetime"] = pd.to_datetime(df_attribution["server_datetime"], errors = "coerce")

In [ ]:
df_event

In [ ]:
df_event.info()

In [ ]:
df_event.head(5)

In [ ]:
df_attribution.info()

In [ ]:
df_attribution.head(5)

## 2.2. Convert csv to parquet

In [ ]:
event_table = pa.Table.from_pandas(df_event)
attribution_table = pa.Table.from_pandas(df_attribution)
event_output_path = "s3://mentha-athena-datastore/sample/event"
attribution_output_path = "s3://mentha-athena-datastore/sample/attribution"

In [ ]:
pq.write_table(event_table, "data/event.parquet")
pq.write_table(attribution_table, "data/attribution.parquet")
# # https://arrow.apache.org/docs/python/generated/pyarrow.parquet.read_metadata.html

In [ ]:
# pq.read_metadata("data/event.parquet")
# read_event_schema = pq.read_schema("data/event.parquet")
# read_event_schema

## 2.3. Upload parquet files to S3

In [ ]:
s3_client.upload_file(Filename="data/event.parquet",
                      Bucket="mentha-athena-datastore",
                      Key="sample/event/event.parquet"
)

In [ ]:
s3_client.upload_file(Filename="data/attribution.parquet",
                      Bucket="mentha-athena-datastore",
                      Key="sample/attribution/attribution.parquet"
)

In [ ]:
# s3 = S3FileSystem()
# pq.write_to_dataset(event_table, root_path=event_output_path, filesystem=s3)
# pq.write_to_dataset(attribution_table, root_path=attribution_output_path, filesystem=s3)

# 3. Athena

## 3.1. Create Athena tables

In [ ]:
from pyathena import connect

In [ ]:
cursor = connect(s3_staging_dir="s3://mentha-athena-query-result/sample",
                 region_name="ap-northeast-2",
                 schema_name="data_store"
).cursor()

In [ ]:
create_event_table = """
CREATE EXTERNAL TABLE IF NOT EXISTS `data_store`.`event` (
  `identity_adid` string,
  `os` string,
  `model` string,
  `country` string,
  `event_name` string,
  `log_id` string,
  `server_datetime` timestamp,
  `quantity` int,
  `price` decimal(10, 1)
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
WITH SERDEPROPERTIES ('serialization.format' = '1')
LOCATION 's3://mentha-athena-datastore/sample/event/'
TBLPROPERTIES ('has_encrypted_data' = 'false');
"""

In [ ]:
create_attribution_table = """
CREATE EXTERNAL TABLE IF NOT EXISTS `data_store`.`attribution` (
  `partner` string,
  `campaign` string,
  `server_datetime` timestamp,
  `tracker_id` string,
  `log_id` string,
  `attribution_type` int,
  `identity_adid` string
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
WITH SERDEPROPERTIES ('serialization.format' = '1')
LOCATION 's3://mentha-athena-datastore/sample/attribution/'
TBLPROPERTIES ('has_encrypted_data' = 'false');
"""

In [ ]:
cursor.execute(create_event_table)
cursor.execute(create_attribution_table)

## 3. Python csv to parquet
- https://www.quora.com/How-do-I-convert-CSV-to-parquet-using-Python-and-without-using-Spark
- Chunk Example: https://stackoverflow.com/questions/26124417/how-to-convert-a-csv-file-to-parquet

In [134]:
## 4. Glue

In [135]:
# {
#     "Version": "2012-10-17",
#     "Statement": [
#         {
#             "Sid": "Statement1",
#             "Effect": "Allow",
#             "Principal": "*",
#             "Action": "s3:*",
#             "Resource": "arn:aws:s3:::mentha-query-datastore/*"
#         }
#     ]
# }

## 4. Read parquet file metadata
- https://arrow.apache.org/docs/python/generated/pyarrow.parquet.read_metadata.html

In [215]:
# s3_client.list_objects(Bucket="mentha-athena-datastore")